Page 1
------

APP
- [ ] récupérer les artistes (names), ID VS names (+ lookup table) ?

UI
- [ ] afficher les artistes (tableaux / liste / autre au choix)
- [ ] sélection des artistes (sélections multiple)
- [ ] button / Post (envoyer la sélection)



Page 2
------

APP
- [ ] récupérer les ids (d'artistes)
- [ ] vectoriser l'utilisateur
- [ ] concaténer l'utilisateur dans la matrice de ratings.
- [ ] fitter le modèle
- [ ] l'ID de l'user:  user_id <-  ?
- [ ] définir la liste d'artistes pour le caclul du score:  artists_ids <- ?
- [ ] faire tourner get_recommandations((user_id, [artists_ids])
- [ ] retourner les meilleurs recommandations ? (limiter le nombre)

UI
- [ ] affichage des recommandations :  nom des artistes (+bonus  afficher le score) 
- [ ] liens vers Page 1

In [61]:
import pandas as pd
import numpy as np

from lightfm import LightFM

from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

from scipy.sparse import coo_matrix, vstack


In [62]:
def Datartist ():
    plays = pd.read_csv('data/lastfm/user_artists.dat', sep='\t')
    artists = pd.read_csv('data/lastfm/artists.dat', sep='\t', usecols=['id','name'])
    # Merge artist and user pref data
    ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
    ap = ap.rename(columns={"weight": "playCount"})
    artist_name = ap['name']
    return ap

In [28]:
ap = Datartist()



In [63]:
def listartist (ap) :
    # Group artist by name
    artist_rank = ap.groupby(['name']) \
        .agg({'userID' : 'count', 'playCount' : 'sum'}) \
        .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
        .sort_values(['totalPlays'], ascending=False)

    artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']

    artist_name = ap['name'].unique()
    
    # Merge into ap matrix
    ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)

    # Preprocessing
    pc = ap.playCount
    play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
    ap = ap.assign(playCountScaled=play_count_scaled)
    #print(ap)

    # Build a user-artist rating matrix 
    ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
    ratings = ratings_df.fillna(0).values
    
    artist_names = ap.sort_values("artistID")["name"].unique()
    
    return ratings, artist_names
    

In [82]:

def buildSM(ratings) : 
    # Build a sparse matrix
    X = csr_matrix(ratings)
    return X




In [96]:
def modeloptimal():
    # définition du model  avec la combinaison de paramètres qui permet d'optimiser le modèle : 0.81 est la suivante 
    modeloptimal = LightFM(learning_schedule = 'adadelta',loss='warp', learning_rate=0.08)
    return modeloptimal

In [89]:
def get_recommendation(model, data, user_id ,artist_names, n):
    n_users, n_items = data.shape
    scores = model.predict(user_id, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    
    return  top_items [:n]

In [79]:
# vectoriser la liste listeName introduite par l'utilisateur

def vectoriserlistArtists(listIndex ,  n) :
    # reprsente la taille de notre vecteur n = 17631
    Z = np.zeros(shape = (1,n))
    for index in listIndex:
        # la valeur 1 sera remplacée par la moyenne playCountScaled de l'artiste en question
        Z[0][index] = 1
    
    return Z



In [68]:
# trouver la liste des index d'artistes introduits par l'utilisateur
def listIndexArtist(listName, artist_name):
    
    listIndex = []
    for name in listName :
        listIndex.append(list(artist_name).index(name))
    return listIndex



In [91]:
ap = Datartist()

# Build a user-artist rating matrix 
ratings, artist_names = listartist (ap)

# Build a sparse matrix
X = buildSM(ratings)



# definition du modèle optimal
model = modeloptimal ()  


# recupérer la liste des artiste à patir du serveur flask
###############


# listNameliste introduite par l'utilisateur : exemple avec les 3 premiers et les 3 derniers
listName = ['MALICE MIZER', 'Diary of Dreams', 'Carpathian Forest','Coptic Rain', 'Oz Alchemist', 'Grzegorz Tomczak']

#  recupérer les index de la liste des artistes introduite par l'user
listIndex = listIndexArtist(listName,artist_name) 

#vectoriser la liste des artistes données par l'utilisateur à partir de la liste d'index
# newVect a la laille n de la matrice sparce X.shape[1] : nbr d'artistes
newVect = vectoriserlistArtists(listIndex , X.shape[1])  

# add the new row to ths space matrix
NewX = vstack([X,newVect])

model.fit(NewX, epochs=10, num_threads=2)

userId = NewX.shape[0]
recommandation = get_recommendation(model, NewX, userId-1 ,artist_names,10)

array(['Radiohead', 'Depeche Mode', 'The Beatles', 'Metallica',
       'Pink Floyd', 'Muse', 'Rammstein', 'Linkin Park', 'Placebo',
       'Lady Gaga'], dtype=object)

In [92]:
NewX.shape

(1893, 17632)

In [97]:
artist_names

array(['MALICE MIZER', 'Diary of Dreams', 'Carpathian Forest', ...,
       'Coptic Rain', 'Oz Alchemist', 'Grzegorz Tomczak'], dtype=object)

In [98]:
listIndex = listIndexArtist(listName,artist_name) 

In [99]:
listIndex

[0, 1, 2, 17629, 17630, 17631]

In [102]:
list(artist_names).index('Diary of Dreams')

1

In [103]:
artist_names

array(['MALICE MIZER', 'Diary of Dreams', 'Carpathian Forest', ...,
       'Coptic Rain', 'Oz Alchemist', 'Grzegorz Tomczak'], dtype=object)

In [119]:
import random  
      
ma_liste = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]  
      
# Obtenir un élément au hasard  
un_element = random.choice(ma_liste)  
      
# Obtenir échantillon de 6 éléments  
echantillon = random.sample(ma_liste, 6)  

In [120]:
echantillon

[6, 1, 7, 2, 8, 11]

In [121]:
def ArtistRandom (liste, nbr):
    # Obtenir échantillon de 6 éléments  
    selection = random.sample(liste, nbr) 
    return selection
                    

In [138]:
l1 = ArtistRandom(list(artist_names), 10)

In [145]:
def recommandation (l,n) :
    listIndex = listIndexArtist(l1,artist_name) 
    newVect = vectoriserlistArtists(listIndex , X.shape[1])
    NewX = vstack([X,newVect])

    model.fit(NewX, epochs=10, num_threads=2)

    userId = NewX.shape[0]
    recommandation = get_recommendation(model, NewX, userId-1 ,artist_names,n)
    return recommandation

In [151]:
l = ArtistRandom(list(artist_names), 10)
print(f" liste random : {l}")
r = recommandation (l,10)
print(f" recommandation : {r}")

 liste random : ['Have Heart', 'Ahmed Bukhatir', 'Мутант Ъхвлам', 'Boom Boom Satellites', 'Portishead', 'E-Rotic', 'Voxtrot', 'Enoch Light', 'Yeong-wook Jo', 'Buried Inside']
 recommandation : ['Deep Purple' 'Sodom' 'Whitesnake' 'Behemoth' 'Megadeth' 'Uriah Heep'
 'Kreator' 'Iron Maiden' 'W.A.S.P.' 'Metin & Kemal Kahraman']


In [152]:
l = ArtistRandom(list(artist_names), 10)
print(f" liste random : {l}")
r = recommandation (l,10)
print(f" recommandation : {r}")

 liste random : ['Älymystö', 'Men at Work', 'Stormlord', 'Mephisto Walz', 'Aura Dione', 'Anberlin', 'The Hush Sound', 'Alberto Iglesias', 'Bonnie Raitt', 'Syrian']
 recommandation : ['Rory Gallagher' 'Beenie Man' 'Henry Mancini' 'Stray Cats' 'Luctus'
 'Sonny Boy Williamson' 'Roy Buchanan' 'Blut aus Nord' 'Wynonie Harris'
 'The Fixx']
